In [1]:
from data_formatting import *

import torch
from torchvision.io import read_image
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.transforms import ToTensor, Lambda, Compose

import os
import pandas as pd
import numpy as np
import matplotlib as plt
import time
import librosa
import soundfile as sf
import warnings
from tqdm.notebook import tqdm
from rich.progress import Progress, TextColumn, BarColumn, TimeRemainingColumn, MofNCompleteColumn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using device: {device}")

warnings.filterwarnings('ignore')

Using device: cuda


In [3]:
calculate_hpss_ratios("E:\\UAV_DISTASIO_DATA\\X\\test","E:\\UAV_DISTASIO_DATA\\X")

Indexing Files:   0%|          | 0/7554 [00:00<?, ?it/s]

Processing files:   0%|          | 0/7554 [00:00<?, ?it/s]

In [17]:
class SpectrogramDataset(Dataset):
    def __init__(self, excel_file, audio_dir, transform=None, target_transform=None):
        self.df = pd.read_excel(excel_file)
        self.audio_dir = audio_dir
        self.transform = transform
        self.target_transform = target_transform
        self.label_map = {1: "Inspired Flight 1200", 2: "DJI Matrice 800", 3: "DJI Phantom 4 Pro v2", 5: "Phantom and Matrice"}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        audio_path = os.path.join(self.audio_dir, self.df.iloc[idx, 0])
        label = self.df.iloc[idx, 1]
        
        waveform, sample_rate = torchaudio.load(audio_path)
        
        # Resample the waveform if necessary
        if sample_rate != 44100:
            waveform = torchaudio.transforms.Resample(sample_rate, 44100)(waveform)
        
        # Convert waveform to spectrogram
        spectrogram = torchaudio.transforms.Spectrogram()(waveform)
        
        # Convert spectrogram to grayscale tensor
        grayscale_spectrogram = spectrogram.mean(dim=0).unsqueeze(0)
        
        if self.transform:
            grayscale_spectrogram = self.transform(grayscale_spectrogram)
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return audio_path, grayscale_spectrogram, torch.tensor(label)
    
    
    def save_spectrograms_as_tensors(self, output_dir):
        os.makedirs(output_dir, exist_ok=True)
        
        for idx in range(len(self)):
            audio_file, spectrogram, label = self[idx]
            
            # Convert label to string
            label_str = self.label_map.get(label.item(), f"Unknown_{label.item()}")
            
            # Generate filename
            filename = f"spectrogram_{audio_file[len("E:_UAV_DISTASIO_DATA_X_test_") :-4]}.pt"
            filepath = os.path.join(output_dir, filename)
            print(filepath)
            
            # Save the spectrogram tensor
            torch.save(spectrogram, filepath)
            
            print(f"Saved spectrogram tensor: {filepath}")

# Usage example
# excel_file = "E:\\UAV_DISTASIO_DATA\\y\\UAV_chunk_labels.xlsx"
# audio_dir = r"E:\UAV_DISTASIO_DATA\X\ESCAPE_FORMAT_ONECHANNEL"
excel_file = "E:\\UAV_DISTASIO_DATA\\y\\UAV_chunk_labels_test.xlsx"
audio_dir = r"E:\UAV_DISTASIO_DATA\X\test"

# Define any transformations if needed
transform = None
label_mapping = {1:0, 2:1, 3:2, 5:3}
target_transform = lambda x: label_mapping[x]

dataset = SpectrogramDataset(excel_file, audio_dir, transform=transform, target_transform=target_transform)

In [18]:
train_size = int(.8 * len(dataset))
test_size = int(.75 * (len(dataset) - train_size))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])
print(train_dataset)
print(train_dataset[0])
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True
    )

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False
)

val_size = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False
)

('E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r10-Matrice+Phantom-North-CrossPath_p02_d306_drone_chunk_14.wav', tensor([[[2.8565e-03, 8.1972e-05, 3.5345e-04,  ..., 1.1654e-02,
          3.9226e-03, 4.5914e-03],
         [6.4902e-04, 1.2088e-03, 2.1260e-03,  ..., 4.2761e-02,
          3.1034e-03, 5.9063e-02],
         [3.6547e-06, 3.3571e-03, 1.1543e-03,  ..., 4.1010e-02,
          1.9025e-02, 2.9090e-02],
         ...,
         [3.0534e-08, 5.0537e-10, 1.9228e-10,  ..., 6.6611e-09,
          3.0035e-09, 3.1270e-07],
         [3.4721e-08, 1.9448e-10, 3.4846e-10,  ..., 6.9277e-09,
          4.4530e-10, 2.3318e-07],
         [3.9093e-08, 1.2790e-11, 6.2475e-11,  ..., 1.9629e-09,
          6.8196e-10, 3.0524e-07]]]), tensor(3))


In [48]:
for audio_path, data, label in train_loader:
    print(audio_path)

('E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r10-Matrice+Phantom-North-CrossPath_p01_d307_drone_chunk_6.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r06-Phantom-North_p01_d304_drone_chunk_4.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r04-Matrice-North_p02_d304_drone_chunk_24.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r06-Phantom-North_p04_d302_drone_chunk_23.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r03-Matrice-South_p01_d306_noDrone_chunk_15.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r09-Matrice+Phantom-South-CrossPath_p02_d307_drone_chunk_11.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r05-Phantom-South_p01_d307_drone_chunk_16.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA2r01-Inspired-South-X_p02_d303_drone_chunk_8.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA2r05-Phantom-South-X_p04_d307_drone_chunk_6.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r06-Phantom-North_p01_d303_drone_chunk_1.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA3r04-Matrice-North_p05_d305_noDrone_chunk_10.wav', 'E:\\UAV_DISTASIO_DATA\\X\\test\\sA

In [57]:
train_dataset[400][1].shape

torch.Size([1, 201, 1103])

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            # Kernel of size (5, 10) to emphasize the harmonic banding drones create in the input spectrograms
            nn.LazyConv2d(4, (5, 10), stride=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.LazyConv2d(8, (5, 10), stride=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d((5, 10), stride=1),

            # Kernel of size (10, 5) on features extracted from the banding
            nn.LazyConv2d(8, (10, 5), stride=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.LazyConv2d(8, (10, 5), stride=1),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d((10, 5), stride=1),

            # Large input from Conv layers requires a larger perceptron for classification
            nn.Flatten(),
            nn.LazyLinear(4096),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(200),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(4)
        )

    def forward(self, x):
        logits = self.seq(x)
        probs = F.softmax(logits, dim=1)
        return probs


In [6]:
def split_seconds(seconds):
    minutes = seconds // 60
    hours = minutes // 60
    days = hours // 24
    return seconds % 60, minutes % 60, hours % 24, days


In [22]:
from tqdm.notebook import tqdm

def main():
    # Define model
    model = CNN()
    # Cuda setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    model = model.to(device)
   
    print(f"Using device: {device}")
    # Optimizer setup
    optimizer = Adam(model.parameters(), lr=1e-3)
   
    # Loss function
    loss_fn = nn.CrossEntropyLoss(reduction="mean")
   
    # Number of epochs
    num_epochs = 64
   
    # Train or load model?
    model.train()
    train_model = True
    print("Training model....")
    start = time.time()
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        total_loss = 0
       
        for batch_idx, (audio_file, images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
           
            # CNN forward pass
            probabilities = model(images)
            loss = loss_fn(probabilities, labels)
            loss.backward()
            optimizer.step()
           
            total_loss += loss.item()
       
        avg_loss = total_loss / len(train_loader)
        tqdm.write(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
       
        torch.save(model.state_dict(), str(start) + '_epoch_' + str(epoch) +".pt")
   
    end = time.time()
    seconds, minutes, hours, days = split_seconds(end - start)
    print(f"Training Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")
   
    # Evaluate model on test data
    model.eval()
    print("Evaluating model....")
    start = time.time()
    num_test = 0
    num_correct = 0
   
    with torch.no_grad():
        for audio_file, images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            probabilities = model(images)
            _, preds = probabilities.max(1)
            num_test += labels.size(0)
            num_correct += preds.eq(labels).sum().item()
   
    print(f"Test accuracy: {num_correct / num_test * 100:.2f}%")
    end = time.time()
    seconds, minutes, hours, days = split_seconds(end - start)
    print(f"Testing Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")

In [23]:
main()

Output()

Using device: cuda
Training model....


Epoch 1/64, Average Loss: 1.4622

Epoch 2/64, Average Loss: 1.4640

Epoch 3/64, Average Loss: 1.4639

...

Epoch 19/64, Average Loss: 1.4642

Epoch 20/64, Average Loss: 1.4640

Epoch 21/64, Average Loss: 1.4641

Epoch 22/64, Average Loss: 1.4642

KeyboardInterrupt: 

In [26]:
# Define model
model = CNN()
# Cuda setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model = model.to(device)

print(f"Using device: {device}")

# Optimizer setup
optimizer = Adam(model.parameters(), lr=1e-3)


state = torch.load("C:\\Users\\Alec\\Desktop\\afrl-uav-detection\\classify\\1719259767.909317_epoch_0.pt")
model.load_state_dict(state)


model.eval()
print("Evaluating model....")
start = time.time()
num_test = 0
num_correct = 0

with torch.no_grad():
    with Progress(
        TextColumn("[progress.description]{task.description}"),
        BarColumn(),
        MofNCompleteColumn(),
        TimeRemainingColumn(),
        refresh_per_second=10
    ) as progress:
        test_task = progress.add_task("[yellow]Testing", total=len(test_loader))
        for audio_file, images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            probabilities = model(images)
            _, preds = probabilities.max(1)
            num_test += labels.size(0)
            num_correct += preds.eq(labels).sum().item()
            progress.update(test_task, advance=1)
            progress.refresh()

print(f"Test accuracy: {num_correct / num_test * 100:.2f}%")
end = time.time()
seconds, minutes, hours, days = split_seconds(end - start)
print(f"Testing Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")

Using device: cuda


Output()

Evaluating model....


Test accuracy: 27.36%


NameError: name 'split_seconds' is not defined

In [71]:
test_dataset[400][1].shape

torch.Size([1, 201, 1103])

In [16]:
# # Save spectrograms as tensors
dataset.save_spectrograms_as_tensors("E:\\UAV_DISTASIO_DATA\\X\\spectrograms")

E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_1.pt
Saved spectrogram tensor: E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_1.pt
E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_10.pt
Saved spectrogram tensor: E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_10.pt
E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_11.pt
Saved spectrogram tensor: E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_11.pt
E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_12.pt
Saved spectrogram tensor: E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_12.pt
E:\UAV_DISTASIO_DATA\X\spectrograms\spectrogram_sA1r01-Inspired-Hover_p01_d302_drone_chunk_2.pt
Saved spectrogram tensor: 

RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 576 vs 470